In [4]:
import pickle as pkl
import networkx as nx
from networkx.readwrite import json_graph
from scipy.sparse.linalg.eigen.arpack import eigsh
from sklearn.metrics import f1_score


import tensorflow as tf
import json
from time import time
import os, copy

flags = tf.app.flags
FLAGS = flags.FLAGS


In [5]:
import numpy as np
import json
import scipy.sparse as sp
import pdb
import sys
sys.setrecursionlimit(99999)


def run_dfs(adj, msk, u, ind, nb_nodes):
    if msk[u] == -1:
        msk[u] = ind
        #for v in range(nb_nodes):
        for v in adj[u,:].nonzero()[1]:
            #if adj[u,v]== 1:
            run_dfs(adj, msk, v, ind, nb_nodes)

# Use depth-first search to split a graph into subgraphs
def dfs_split(adj):
    # Assume adj is of shape [nb_nodes, nb_nodes]
    nb_nodes = adj.shape[0]
    ret = np.full(nb_nodes, -1, dtype=np.int32)

    graph_id = 0

    for i in range(nb_nodes):
        if ret[i] == -1:
            run_dfs(adj, ret, i, graph_id, nb_nodes)
            graph_id += 1

    return ret

def test(adj, mapping):
    nb_nodes = adj.shape[0]
    for i in range(nb_nodes):
        #for j in range(nb_nodes):
        for j in adj[i, :].nonzero()[1]:
            if mapping[i] != mapping[j]:
              #  if adj[i,j] == 1:
                 return False
    return True



def find_split(adj, mapping, ds_label):
    nb_nodes = adj.shape[0]
    dict_splits={}
    for i in range(nb_nodes):
        #for j in range(nb_nodes):
        for j in adj[i, :].nonzero()[1]:
            if mapping[i]==0 or mapping[j]==0:
                dict_splits[0]=None
            elif mapping[i] == mapping[j]:
                if ds_label[i]['val'] == ds_label[j]['val'] and ds_label[i]['test'] == ds_label[j]['test']:

                    if mapping[i] not in dict_splits.keys():
                        if ds_label[i]['val']:
                            dict_splits[mapping[i]] = 'val'

                        elif ds_label[i]['test']:
                            dict_splits[mapping[i]]='test'

                        else:
                            dict_splits[mapping[i]] = 'train'

                    else:
                        if ds_label[i]['test']:
                            ind_label='test'
                        elif ds_label[i]['val']:
                            ind_label='val'
                        else:
                            ind_label='train'
                        if dict_splits[mapping[i]]!= ind_label:
                            print ('inconsistent labels within a graph exiting!!!')
                            return None
                else:
                    print ('label of both nodes different, exiting!!')
                    return None
    return dict_splits




def process_p2p():


    print ('Loading G...')
    with open('ppi/ppi-G.json') as jsonfile:
        g_data = json.load(jsonfile)
    print (len(g_data))
    G = json_graph.node_link_graph(g_data)

    #Extracting adjacency matrix
    adj=nx.adjacency_matrix(G)

    prev_key=''
    for key, value in g_data.items():
        if prev_key!=key:
            print (key)
            prev_key=key

    print ('Loading id_map...')
    with open('ppi/ppi-id_map.json') as jsonfile:
        id_map = json.load(jsonfile)
    print (len(id_map))

    id_map = {int(k):int(v) for k,v in id_map.items()}
    for key, value in id_map.items():
        id_map[key]=[value]
    print (len(id_map))

    print ('Loading features...')
    features_=np.load('ppi/ppi-feats.npy')
    print (features_.shape)

    #standarizing features
    from sklearn.preprocessing import StandardScaler

    train_ids = np.array([id_map[n] for n in G.nodes() if not G.node[n]['val'] and not G.node[n]['test']])
    train_feats = features_[train_ids[:,0]]
    scaler = StandardScaler()
    scaler.fit(train_feats)
    features_ = scaler.transform(features_)

    features = sp.csr_matrix(features_).tolil()


    print ('Loading class_map...')
    class_map = {}
    with open('ppi/ppi-class_map.json') as jsonfile:
        class_map = json.load(jsonfile)
    print (len(class_map))
    
    #pdb.set_trace()
    #Split graph into sub-graphs
    print ('Splitting graph...')
    splits=dfs_split(adj)

    #Rearrange sub-graph index and append sub-graphs with 1 or 2 nodes to bigger sub-graphs
    print ('Re-arranging sub-graph IDs...')
    list_splits=splits.tolist()
    group_inc=1

    for i in range(np.max(list_splits)+1):
        if list_splits.count(i)>=3:
            splits[np.array(list_splits) == i] =group_inc
            group_inc+=1
        else:
            #splits[np.array(list_splits) == i] = 0
            ind_nodes=np.argwhere(np.array(list_splits) == i)
            ind_nodes=ind_nodes[:,0].tolist()
            split=None
            
            for ind_node in ind_nodes:
                if g_data['nodes'][ind_node]['val']:
                    if split is None or split=='val':
                        splits[np.array(list_splits) == i] = 21
                        split='val'
                    else:
                        raise ValueError('new node is VAL but previously was {}'.format(split))
                elif g_data['nodes'][ind_node]['test']:
                    if split is None or split=='test':
                        splits[np.array(list_splits) == i] = 23
                        split='test'
                    else:
                        raise ValueError('new node is TEST but previously was {}'.format(split))
                else:
                    if split is None or split == 'train':
                        splits[np.array(list_splits) == i] = 1
                        split='train'
                    else:
                        pdb.set_trace()
                        raise ValueError('new node is TRAIN but previously was {}'.format(split))

    #counting number of nodes per sub-graph
    list_splits=splits.tolist()
    nodes_per_graph=[]
    for i in range(1,np.max(list_splits) + 1):
        nodes_per_graph.append(list_splits.count(i))

    #Splitting adj matrix into sub-graphs
    subgraph_nodes=np.max(nodes_per_graph)
    adj_sub=np.empty((len(nodes_per_graph), subgraph_nodes, subgraph_nodes))
    feat_sub = np.empty((len(nodes_per_graph), subgraph_nodes, features.shape[1]))
    labels_sub = np.empty((len(nodes_per_graph), subgraph_nodes, 121))

    for i in range(1, np.max(list_splits) + 1):
        #Creating same size sub-graphs
        indexes = np.where(splits == i)[0]
        subgraph_=adj[indexes,:][:,indexes]

        if subgraph_.shape[0]<subgraph_nodes or subgraph_.shape[1]<subgraph_nodes:
            subgraph=np.identity(subgraph_nodes)
            feats=np.zeros([subgraph_nodes, features.shape[1]])
            labels=np.zeros([subgraph_nodes,121])
            #adj
            subgraph = sp.csr_matrix(subgraph).tolil()
            subgraph[0:subgraph_.shape[0],0:subgraph_.shape[1]]=subgraph_
            adj_sub[i-1,:,:]=subgraph.todense()
            #feats
            feats[0:len(indexes)]=features[indexes,:].todense()
            feat_sub[i-1,:,:]=feats
            #labels
            for j,node in enumerate(indexes):
                labels[j,:]=np.array(class_map[str(node)])
            labels[indexes.shape[0]:subgraph_nodes,:]=np.zeros([121])
            labels_sub[i - 1, :, :] = labels

        else:
            adj_sub[i - 1, :, :] = subgraph_.todense()
            feat_sub[i - 1, :, :]=features[indexes,:].todense()
            for j,node in enumerate(indexes):
                labels[j,:]=np.array(class_map[str(node)])
            labels_sub[i-1, :, :] = labels

    # Get relation between id sub-graph and tran,val or test set
    dict_splits = find_split(adj, splits, g_data['nodes'])

    # Testing if sub graphs are isolated
    print ('Are sub-graphs isolated?')
    print (test(adj, splits))

    #Splitting tensors into train,val and test
    train_split=[]
    val_split=[]
    test_split=[]
    for key, value in dict_splits.items():
        if dict_splits[key]=='train':
            train_split.append(int(key)-1)
        elif dict_splits[key] == 'val':
            val_split.append(int(key)-1)
        elif dict_splits[key] == 'test':
            test_split.append(int(key)-1)

    train_adj=adj_sub[train_split,:,:]
    val_adj=adj_sub[val_split,:,:]
    test_adj=adj_sub[test_split,:,:]

    train_feat=feat_sub[train_split,:,:]
    val_feat = feat_sub[val_split, :, :]
    test_feat = feat_sub[test_split, :, :]

    train_labels = labels_sub[train_split, :, :]
    val_labels = labels_sub[val_split, :, :]
    test_labels = labels_sub[test_split, :, :]

    train_nodes=np.array(nodes_per_graph[train_split[0]:train_split[-1]+1])
    val_nodes = np.array(nodes_per_graph[val_split[0]:val_split[-1]+1])
    test_nodes = np.array(nodes_per_graph[test_split[0]:test_split[-1]+1])


    #Masks with ones

    tr_msk = np.zeros((len(nodes_per_graph[train_split[0]:train_split[-1]+1]), subgraph_nodes))
    vl_msk = np.zeros((len(nodes_per_graph[val_split[0]:val_split[-1] + 1]), subgraph_nodes))
    ts_msk = np.zeros((len(nodes_per_graph[test_split[0]:test_split[-1]+1]), subgraph_nodes))

    for i in range(len(train_nodes)):
        for j in range(train_nodes[i]):
            tr_msk[i][j] = 1

    for i in range(len(val_nodes)):
        for j in range(val_nodes[i]):
            vl_msk[i][j] = 1

    for i in range(len(test_nodes)):
        for j in range(test_nodes[i]):
            ts_msk[i][j] = 1

    return train_adj, val_adj, test_adj, train_feat, val_feat, test_feat, train_labels, val_labels, test_labels, train_nodes, val_nodes, test_nodes, tr_msk, vl_msk, ts_msk

In [6]:
def sparse_to_tuple(sparse_mx):
    """Convert sparse matrix to tuple representation."""
    def to_tuple(mx):
        if not sp.isspmatrix_coo(mx):
            mx = mx.tocoo()
        coords = np.vstack((mx.row, mx.col)).transpose()
        values = mx.data
        shape = mx.shape
        return coords, values, shape

    if isinstance(sparse_mx, list):
        for i in range(len(sparse_mx)):
            sparse_mx[i] = to_tuple(sparse_mx[i])
    else:
        sparse_mx = to_tuple(sparse_mx)

    return sparse_mx


def preprocess_features(features):
    """Row-normalize feature matrix and convert to tuple representation"""
    rowsum = np.array(features.sum(1))
    r_inv = np.power(rowsum, -1).flatten()
    r_inv[np.isinf(r_inv)] = 0.
    r_mat_inv = sp.diags(r_inv)
    features = r_mat_inv.dot(features)
    return sparse_to_tuple(sp.csr_matrix(features))


def preprocess_adj_bias(adj):
    num_nodes = adj.shape[0]
    adj = adj + sp.eye(num_nodes)  # self-loop
    adj[adj > 0.0] = 1.0
    return sparse_to_tuple(sp.csr_matrix(adj))

In [17]:
def micro_f1(logits, labels, mask):
    """Accuracy with masking."""
    predicted = tf.round(tf.nn.sigmoid(logits))

    # Use integers to avoid any nasty FP behaviour
    predicted = tf.cast(predicted, dtype=tf.int32)
    labels = tf.cast(labels, dtype=tf.int32)
    mask = tf.cast(mask, dtype=tf.int32)
    
    # expand the mask so that broadcasting works ([nb_nodes, 1])
#     mask = tf.expand_dims(mask, -1)


    # Count true positives, true negatives, false positives and false negatives.
    tp = tf.count_nonzero(predicted * labels * mask)
    tn = tf.count_nonzero((predicted - 1) * (labels - 1) * mask)
    fp = tf.count_nonzero(predicted * (labels - 1) * mask)
    fn = tf.count_nonzero((predicted - 1) * labels * mask)

    # Calculate accuracy, precision, recall and F1 score.
    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    fmeasure = (2 * precision * recall) / (precision + recall)
    fmeasure = tf.cast(fmeasure, tf.float32)
    return fmeasure





def masked_sigmoid_cross_entropy(logits, labels, mask):
    """Softmax cross-entropy loss with masking."""
    labels = tf.cast(labels, dtype=tf.float32)
    loss = tf.nn.sigmoid_cross_entropy_with_logits(logits=logits, labels=labels)
    loss=tf.reduce_mean(loss,axis=1)
    mask = tf.cast(mask, dtype=tf.float32)
    mask /= tf.reduce_mean(mask)
    loss *= mask
    return tf.reduce_mean(loss)


def sp_attn_head(seq, in_sz, out_sz, adj_mat, activation, nb_nodes, 
                 in_drop=0.0, coef_drop=0.0, residual=True, sparse=True):
    with tf.name_scope('sp_attn'):
        if in_drop != 0.0:
            seq = tf.nn.dropout(seq, 1.0 - in_drop)
        w=glorot([in_sz, out_sz])
        b=zeros([out_sz])
        if sparse:
            seq_fts = dot(seq, w, sparse=True) # activation
        else:
            seq_fts = dot(seq, w, sparse=False)


        # simplest self-attention possible
        seq_fts=tf.expand_dims(seq_fts, axis=0)
        f_1 = tf.layers.conv1d(seq_fts, 1, 1)
        f_2 = tf.layers.conv1d(seq_fts, 1, 1)

        f_1 = tf.reshape(f_1, (nb_nodes, 1)) 
        f_2 = tf.reshape(f_2, (nb_nodes, 1))

        f_1 = adj_mat*f_1
        f_2 = adj_mat * tf.transpose(f_2, [1,0])

        logits = tf.sparse_add(f_1, f_2)
        lrelu = tf.SparseTensor(indices=logits.indices, 
                values=tf.nn.leaky_relu(logits.values), 
                dense_shape=logits.dense_shape)
        coefs = tf.sparse_softmax(lrelu)

        if coef_drop != 0.0:
            coefs = tf.SparseTensor(indices=coefs.indices,
                    values=tf.nn.dropout(coefs.values, 1.0 - coef_drop),
                    dense_shape=coefs.dense_shape)
        if in_drop != 0.0:
            seq_fts = tf.nn.dropout(seq_fts, 1.0 - in_drop)

        # As tf.sparse_tensor_dense_matmul expects its arguments to have rank-2,
        # here we make an assumption that our input is of batch size 1, and reshape appropriately.
        # The method will fail in all other cases!
        coefs = tf.sparse_reshape(coefs, [nb_nodes, nb_nodes])
        seq_fts = tf.squeeze(seq_fts)
        vals = tf.sparse_tensor_dense_matmul(coefs, seq_fts)
        ret = vals+b

        # residual connection
        if residual:
            if in_sz != out_sz:
                ww=glorot([in_sz, out_sz])
                if sparse:
                    ret = ret + dot(seq, ww, sparse=True) # activation
                else:
                    ret = ret + dot(seq, ww, sparse=False)
            else:
                ret = ret + seq

        return activation(ret)  # activation
    
    
def dot(x, y, sparse=False):
    """Wrapper for tf.matmul (sparse vs dense)."""
    if sparse:
        res = tf.sparse_tensor_dense_matmul(x, y)
    else:
        res = tf.matmul(x, y)
    return res


def masked_softmax_cross_entropy(preds, labels, mask):
    """Softmax cross-entropy loss with masking."""
    loss = -tf.reduce_sum(labels*tf.log(tf.nn.softmax(preds)+1e-7), axis=1)
    mask = tf.cast(mask, dtype=tf.float32)
    mask /= tf.reduce_mean(mask)
    loss *= mask
    return tf.reduce_mean(loss)


def masked_accuracy(preds, labels, mask):
    """Accuracy with masking."""
    correct_prediction = tf.equal(tf.argmax(preds, 1), tf.argmax(labels, 1))
    accuracy_all = tf.cast(correct_prediction, tf.float32)
    mask = tf.cast(mask, dtype=tf.float32)
    mask /= tf.reduce_mean(mask)
    accuracy_all *= mask
    return tf.reduce_mean(accuracy_all)


def glorot(shape, name=None):
    """Glorot & Bengio (AISTATS 2010) init."""
    init_range = np.sqrt(6.0/(shape[0]+shape[1]))
    initial = tf.random_uniform(shape, minval=-init_range, maxval=init_range, dtype=tf.float32)
    return tf.Variable(initial, name=name)


def zeros(shape, name=None):
    """All zeros."""
    initial = tf.zeros(shape, dtype=tf.float32)
    return tf.Variable(initial, name=name)



In [15]:
class GAT_Sparse_Inductive():
    def __init__(self, hid_units, n_heads, n_nodes, f_dimension, nb_classes, lr):
                    
        
        
        self.X = tf.sparse_placeholder(tf.float32,  name='X')
        self.y = tf.placeholder('float32', name='y')
        self.mask = tf.placeholder('float32', name='Mask')
        self.adj = tf.sparse_placeholder(tf.float32, name='ADJ')

        
        ########### Input Layer #################
        attns = []
        for _ in range(n_heads[0]):
            attns.append(sp_attn_head(self.X, in_sz=f_dimension , adj_mat=self.adj,out_sz=hid_units[0], activation=tf.nn.elu, nb_nodes=n_nodes,
                in_drop=0, coef_drop=0, residual=True, sparse=True))
            
        h_1 = tf.concat(attns, axis=-1)
        
        ########### Hidden Layer #################

        for i in range(1, len(hid_units)):
            h_old = h_1
            attns = []
            for _ in range(n_heads[i]):
                attns.append(sp_attn_head(h_1, in_sz=hid_units[i-1]*n_heads[i-1] ,adj_mat=self.adj,out_sz=hid_units[i], activation=tf.nn.elu, nb_nodes=n_nodes,
                    in_drop=0, coef_drop=0, residual=True, sparse=False))
            h_1 = tf.concat(attns, axis=-1)
        
        ########### Output Layer #################
        out = []
        for _ in range(n_heads[-1]):
            out.append(sp_attn_head(h_1, in_sz=hid_units[-2]*n_heads[-2] ,adj_mat=self.adj, out_sz=nb_classes, activation=lambda x: x, nb_nodes=n_nodes,
                    in_drop=0, coef_drop=0, residual=True, sparse=False))
        logits = tf.add_n(out) / n_heads[-1]
        self.logits=logits
        
        
        
        
        self.loss(lr)
        
    def accuracy(self, X, Y, mask, adj):
        """Get accuracy"""
        return self.sess.run(self.acc,
                             feed_dict={self.adj: adj, self.X: X, self.y: Y, self.mask: mask})

    


    def train(self, batch_xs, batch_ys, mask, adj):
        _ = self.sess.run(self.trains, feed_dict={ self.y: batch_ys, self.adj: adj
            ,self.X: batch_xs, self.mask: mask})
        
        

    def loss(self, lr):
        self.cost = masked_sigmoid_cross_entropy(self.logits, self.y, self.mask) 

      
        self.optimizer = tf.train.AdamOptimizer(learning_rate=lr)
        self.trains = self.optimizer.minimize(self.cost)
        self.acc = micro_f1(self.logits, self.y, self.mask)

        self.sess = tf.Session()
        self.sess.run(tf.global_variables_initializer())

In [10]:
train_adj, val_adj, test_adj, train_feat, val_feat, test_feat, train_labels, val_labels, test_labels, train_nodes, val_nodes, test_nodes, tr_msk, vl_msk, ts_msk =  process_p2p() 

Loading G...
5
directed
graph
nodes
links
multigraph
Loading id_map...
56944
56944
Loading features...
(56944, 50)
Loading class_map...
56944
Splitting graph...
Re-arranging sub-graph IDs...
Are sub-graphs isolated?
True


In [11]:
print('feature shape: ', train_feat[0].shape)
print('adj shape: ', train_adj[0].shape)
print('y_train shape: ', train_labels[0].shape)
print('test_set: ', np.sum(ts_msk))
print('train_set: ',np.sum(tr_msk))
print()

feature shape:  (3480, 50)
adj shape:  (3480, 3480)
y_train shape:  (3480, 121)
test_set:  5524.0
train_set:  44906.0



In [12]:
train_f=[]
train_a=[]
for i in range(20):
    train_f.append(preprocess_features(train_feat[i]))
    train_a.append(preprocess_adj_bias(train_adj[i]))
    
test_f=[]
test_a=[]  
for i in range(2):
    test_f.append(preprocess_features(test_feat[i]))
    test_a.append(preprocess_adj_bias(test_adj[i]))

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:23: RuntimeWarning: divide by zero encountered in power


In [61]:
tf.reset_default_graph()
model=GAT_Sparse_Inductive([256, 256], [4,4,6], 3480, 50, 121, 5e-3)
best=0
for epoch in range(1000):
    for i in range(20):
        avg_cost=0
        batch_xs, batch_ys, adj, mask, adj_for_ae = train_f[i*1:(i+1)*1], train_labels[i*1:(i+1)*1],  train_a[i*1:(i+1)*1], tr_msk[i*1:(i+1)*1], train_adj[i*1:(i+1)*1]
        model.train(batch_xs[0], batch_ys[0], mask[0][:,np.newaxis], adj[0])
    if epoch%50==0:
        test_acc1=model.accuracy(test_f[0], test_labels[0], ts_msk[0][:,np.newaxis], test_a[0])
        test_acc2=model.accuracy(test_f[1], test_labels[1], ts_msk[1][:,np.newaxis], test_a[1])
        print((test_acc1+test_acc2)/2) 

IndexError: list index out of range